# Presets

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Thu_Jun_11_22:26:38_PDT_2020
Cuda compilation tools, release 11.0, V11.0.194
Build cuda_11.0_bu.TC445_37.28540450_0


In [ ]:
!pip install --user tensorflow==2.3.0

In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
# Clone the tensorflow models repository if it doesn't already exist

import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install --user .

# Dataset

Download the [MSCOCO](https://cocodataset.org/#home) dataset and focus only on person annotations and images since we want to train for person detection. 

In [ ]:
# Download coco dataset and prepare records
%cd ~/models/research/
!bash object_detection/dataset_tools/download_and_preprocess_mscoco.sh ~/data/mscoco

# Training

Reference our configured pipeline and output of the training

In [ ]:
CONFIG="models/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_96x96_coco17_tpu-8.config"
MODEL_OUT="models/ssd_mobilenet_v2_fpn"

In [ ]:
# Run training
%cd ~/
!python models/research/object_detection/model_main_tf2.py \
--model_dir=$MODEL_OUT \
--num_train_steps=10000 \
--sample_1_of_n_eval_examples=1 \
--pipeline_config_path=$CONFIG \
--alsologtostderr

# Test model

# Export model

In [38]:
RESULTS="models/results/saved_model"
MODEL_NO_QUANT_TFLITE="models/results/model_no_quant.tflite"
MODEL_MICRO="models/results/model.cc"

In [ ]:
# Export checkpoint to .pb
!python models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path=$CONFIG \
    --trained_checkpoint_dir=$MODEL_OUT \
    --output_directory=$RESULTS

In [36]:
import tensorflow as tf

# Convert the model to the TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model(RESULTS)
model_no_quant_tflite = converter.convert()

open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

516

In [40]:
# Convert to a C source file
!apt-get update && apt-get -qq install xxd
!xxd -i {MODEL_NO_QUANT_TFLITE} > {MODEL_MICRO}
REPLACE_TEXT = MODEL_NO_QUANT_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_MICRO}

Reading package lists... Done
W: chmod 0700 of directory /var/lib/apt/lists/partial failed - SetupAPTPartialDirectory (1: Operation not permitted)
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
